In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import label_binarize
import string
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
import re
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt
from itertools import cycle
from scipy import interp

import requests
import io

from sklearn.model_selection import GridSearchCV

import mlflow
import mlflow.sklearn
from urllib.parse import urlparse

# Own Functions
import NLP_Functions as Func
"""
import imp
imp.reload(Func)
"""

'\nimport imp\nimp.reload(Func)\n'

In [2]:
                        # Read CSVs

X_train, Y_train, X_test, Y_test = Func.readCSVs_URL(
    "https://raw.githubusercontent.com/SoniaLei/nlp-web-scrapping/development/data/raw/tweets-train.csv",
    "https://raw.githubusercontent.com/SoniaLei/nlp-web-scrapping/development/data/raw/tweets-test.csv"
)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating Vectorisers + List

bow_vector = CountVectorizer(tokenizer = Func.text_cleaner, ngram_range=(1,1))

tfidf_vector = TfidfVectorizer(tokenizer = Func.text_cleaner)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Build Classifiers + List
    
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

# Logistic Regression - OvR

LogReg = LogisticRegression(max_iter=100)

ovr = OneVsRestClassifier(LogReg)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

# Random Forest

rfc = RandomForestClassifier(n_estimators = 10, n_jobs=-1)

print(Func.SAM_Name(ovr),"\n")
print(Func.SAM_Name(rfc))
print("#-" * 50)
print(ovr,"\n")
print(rfc)
print("#-" * 50)

LogisticRegression (OneVsRestClassifier) 

RandomForestClassifier
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=N

In [3]:
tuned_parameters = [
    {'vectorizer': [bow_vector, tfidf_vector],
     'classifier': [ovr, rfc]}
]
# A list of dictionaries for the Grid_Search to loop through
    # 2 different vectorisers
    # 2 different SAMs

"""
tuned_parameters = [
    {'classifier': [rfc],
    'classifier__n_estimators': [10, 15, 20]}
]

tuned_parameters = [
    {'vectorizer': [bow_vector, tfidf_vector]},
    {'classifier': [rfc],
    'classifier__n_estimators': [10, 15, 20]},
    {'classifier': [ovr],
    'classifer__estimator__penalty': ['l2', 'none']}
]
"""

"\ntuned_parameters = [\n    {'classifier': [rfc],\n    'classifier__n_estimators': [10, 15, 20]}\n]\n\ntuned_parameters = [\n    {'vectorizer': [bow_vector, tfidf_vector]},\n    {'classifier': [rfc],\n    'classifier__n_estimators': [10, 15, 20]},\n    {'classifier': [ovr],\n    'classifer__estimator__penalty': ['l2', 'none']}\n]\n"

In [8]:
pipe = Pipeline([('vectorizer', bow_vector)
                 ,('classifier', None)])

scores = ['precision'] #, 'recall']
# This asks what scoring metric you want the models to be calculated aganist
    # Which model has the best precision / recall score
    
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Use Grid_Search on Parameters

for score in scores:
    
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        pipe, param_grid=tuned_parameters, scoring = '%s_macro' % score, cv = 5, n_jobs = -1
    )
    # Initialising the GridSeachCV
        # Looping through the different parameters
    
    clf.fit(X_train, Y_train)
    # Fitting the model
    
    print("Best parameters set found on development set:")
    print()

    #print(Func.Names_GridSearch_BestParams(tuned_parameters, clf.best_params_))
    print(clf.best_params_)
        # Best Parameters for the 'score' the model is looking at
    print()

    print("Grid scores on development set:")
    print()

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Logging the Grid_Search's results with ML Flow
    
    exp_id = Func.GetExper_CreateExper("Grid_Search_Test")
    # Set experiment to log data into
    
    print("Exp_id: " + str(exp_id))

    mlflow.set_tracking_uri("file:///C:/Users/nathi_000/Desktop/Python Files/NLP Project/nlp-web-scrapping/mlruns")
    # Set location for data to be stored
    
    means = clf.cv_results_['mean_test_score']
    # Mean_test_score for the scoring parameter the model is on

    stds = clf.cv_results_['std_test_score']
    # Std_test_score for the scoring parameter the model is on
    
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    # Looping through the Mean_test_score and standard_test_score for each of the parameters:
        
        with mlflow.start_run(experiment_id = exp_id):
            
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params)) #Func.Names_GridSearch_BestParams(tuned_parameters, params)))
            
            # Parameters
            mlflow.log_param("SAM", params)
            # Logging the SAM details for each model in the Grid_Search
            
            # Metrics
            mlflow.log_metric("Mean_test_score", mean)
            # Logging the mean_test_score for each model in the Grid_Search

            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow

                mlflow.sklearn.log_model(pipe, "model", registered_model_name="LogisticRegression")

            else:
                mlflow.sklearn.log_model(pipe, "model")
                # Logging the model in ML Flow

    print()

    print("Detailed classification report:")
    print()

    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()

    y_pred = clf.predict(X_test)
    print(classification_report(Y_test, y_pred))
    print(accuracy_score(Y_test, y_pred))
    print(precision_score(Y_test, y_pred, average = 'macro'))
    # Report of all the scores for the model with the best parameters

# Tuning hyper-parameters for precision



C:\Users\nathi_000\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Best parameters set found on development set:

{'classifier': OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None), 'vectorizer': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
  

In [5]:
print(clf.best_params_['classifier'].n_estimators)

#print(str(clf.best_params_['classifier']['n_estimators']))

AttributeError: 'OneVsRestClassifier' object has no attribute 'n_estimators'